# Basic Alhazen Agent / Chatbot.

> Building a simple Gradio application to allow the user to chat to Alhazen. 

In [1]:
#| default_exp apps.marimo_chat

In [2]:
#| export

from alhazen.core import MODEL_TYPE, PromptTemplateRegistry, load_alhazen_tool_environment, get_langchain_chatmodel
from alhazen.schema_sqla import *
from alhazen.tools.basic import IntrospectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database, list_databases

import click

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, AgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.output_parsers.json import parse_json_markdown
from langchain.pydantic_v1 import BaseModel
from langchain.tools.render import render_text_description, render_text_description_and_args
from langchain.agents.output_parsers.json import JSONAgentOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool

from langchain.prompts import ChatPromptTemplate

import marimo as mo

import os
import pandas as pd
from pathlib import Path
import re
import requests

from sqlalchemy import create_engine, text, exists, func, or_, and_, not_, desc, asc
from sqlalchemy.orm import sessionmaker, aliased

import sys
from typing import Dict, List, Optional, Sequence, Union, Any
from uuid import UUID

/Users/gully.burns/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export 
# Using Aliases like this massively simplifies the use of SQLAlchemy
IR = aliased(InformationResource)

SKC = aliased(ScientificKnowledgeCollection)
SKC_HM = aliased(ScientificKnowledgeCollectionHasMembers)
SKE = aliased(ScientificKnowledgeExpression)
SKE_XREF = aliased(ScientificKnowledgeExpressionXref)
SKE_IRI = aliased(ScientificKnowledgeExpressionIri)
SKE_HR = aliased(ScientificKnowledgeExpressionHasRepresentation)
SKE_MO = aliased(ScientificKnowledgeExpressionMemberOf)
SKI = aliased(ScientificKnowledgeItem)
SKI_HP = aliased(ScientificKnowledgeItemHasPart)
SKF = aliased(ScientificKnowledgeFragment)

N = aliased(Note)
NIA = aliased(NoteIsAbout)
SKC_HN = aliased(ScientificKnowledgeCollectionHasNotes)
SKE_HN = aliased(ScientificKnowledgeExpressionHasNotes)
SKI_HN = aliased(ScientificKnowledgeItemHasNotes)
SKF_HN = aliased(ScientificKnowledgeFragmentHasNotes)

In [2]:
#| export 

def mo_list_databases_dropdown():
    """
    Adds a dropdown list of available databases as a.
    """
    dbn = list_databases()
    dropdown = mo.ui.dropdown(
        options={n:n for n in dbn},
        value=dbn[0],
        label='Available Databases: '
        )
    return dropdown

class AlhazenAgentMarmimoDashboardElements:
    '''Provide access to an Alhazen agent via a simple chat interface with Marimo.'''

    def __init__(self, agent_llm, tool_llm, sml=None, verbose=True, return_intermediate_steps=True):

        loc, db_name = load_alhazen_tool_environment()

        self.db = Ceifns_LiteratureDb(loc=loc, name=db_name)
        self.agent_llm  = agent_llm
        self.tool_llm = tool_llm
        self.sml = sml
        self.chat_messages = []
        self.command_ta = mo.ui.text_area(placeholder='Enter command for Alhazen...', label="Command", full_width=True)
        self.command_form = self.command_ta.form()

        self.tk = AlhazenToolkit(db=self.db, llm=self.tool_llm)
        self.tools = self.tk.get_tools()

        pts = PromptTemplateRegistry()
        pts.load_prompts_from_yaml('alhazen_base.yaml')

        self.prompt = pts.get_prompt_template('structured chat agent').generate_chat_prompt_template()
        
        agent = create_structured_chat_agent_withFixes(self.agent_llm, self.tools, self.prompt)
        #introspection_tool = [t for t in self.tools if isinstance(t, IntrospectionTool)][0]
        #introspection_tool._set_agent(agent)

        self.agent_executor = AgentExecutor(agent=agent, 
                                            tools=self.tools,
                                            verbose=verbose, 
                                            return_intermediate_steps=return_intermediate_steps)

    def get_introspection_tool(self):
        return [t for t in self.tools if isinstance(t, IntrospectionTool)][0]

    # We need to add these input/output schemas because the current AgentExecutor
    # is lacking in schemas.
    class ChatRequest(BaseModel):
        question: str
        chat_history: Optional[List[Dict[str, str]]]

    class Input(BaseModel):
        input: str

    class Output(BaseModel):
        output: str
    
    def mo_collection_item_counts_primary_research(self):
        """
        Returns a table of the counts of primary research articles and preprints in each collection.
        """
        q = self.db.session.query(SKC.id, SKC.name, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id) \
            .filter(or_(SKE.type=='ScientificPrimaryResearchArticle', SKE.type=='ScientificPrimaryResearchPreprint'))
        df1 = pd.DataFrame(q.all(), columns=['ID', 'collection', 'doi', 'type'])    
        df2 = df1.pivot_table(index=['ID', 'collection'], 
                      columns='type', values='doi', 
                      aggfunc=lambda x: len(x.unique()))
        return mo.ui.table(df2, selection='single')

    def mo_collection_item_counts(self):
        """
        Returns a table of the counts of primary research articles and preprints in each collection.
        """
        q = self.db.session.query(SKC.id, SKC.name, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id)
        df1 = pd.DataFrame(q.all(), columns=['ID', 'collection', 'doi', 'type'])    
        df2 = df1.pivot_table(index=['ID', 'collection'], 
                      columns='type', values='doi', 
                      aggfunc=lambda x: len(x.unique()))
        return mo.ui.table(df2, selection='single')
    
    def mo_chatbot_control(self):
        chatbot = []
        for c in self.chat_messages:
            if c[0] == 'system':            
                m = mo.md(c[1]).left()
            elif c[0] == 'user':            
                m = mo.md(c[1]).right()
            chatbot.append(m)
        chatbot.append(self.command_form)
        return mo.vstack(chatbot)
    
    def invoke_command(self, command):
        if command is None or len(command) == 0:
            return self.mo_chatbot_control()
        self.chat_messages.append(('user', command))
        try:
            response = self.agent_executor.invoke({'input': command})
            self.chat_messages.append(('system', response.get('answer')))
        except Exception as e:
            self.chat_messages.append(('system', f'Error: {e}'))

In [ ]:
#| export 
def create_structured_chat_agent_withFixes(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """modified from `create_structured_chat_agent` to deal with escaped underscore characters."""

    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
        prompt.input_variables
    )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=render_text_description_and_args(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm_with_stop = llm.bind(stop=["Observation"])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | JSONAgentOutputParser_withFixes()
    )
    return agent

class JSONAgentOutputParser_withFixes(AgentOutputParser):
    """modified from `JSONAgentOutputParser` to deal with escaped underscore characters."""

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        try:
            text = text.strip()

            # Hack to remove escaped underscores.
            text = re.sub('\\\\_', '_', text)
            text = re.sub('\\\\_', '_', text)

            # Hack to look for the JSON code fragment 
            # inside other text inside the 
            m = re.search('.*?([\[\{](.|\n)*[\}\]]).*?', text, flags=re.M)
            if m:
                text = m.group(1)

            response = parse_json_markdown(text)
            if isinstance(response, list):
                # gpt turbo frequently ignores the directive to emit a single action
                response = response[0]
            if response["action"] == "Final Answer":
                return AgentFinish({"output": response["action_input"]}, text)
            else:
                return AgentAction(
                    response["action"], response.get("action_input", {}), text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e

    @property
    def _type(self) -> str:
        return "json-agent"

In [1]:
#| export
@click.command()
@click.option('--loc', default=None, help='Where on disk to put local files')
@click.option('--db_name', prompt=None, help='The name of the database being used.')
def alhazen_chatbot(loc, db_name):
    """Runs the Alhazen Chatbot."""
    if loc:
        os.environ['LOCAL_FILE_PATH'] = loc
    if db_name:
        os.environ['ALHAZEN_DB_NAME'] = db_name
    cb = AlhazenAgentGradioChatBot()
    cb.run_gradio()


NameError: name 'click' is not defined

In [8]:
#| export
#|eval: false

if __name__ == '__main__':
    alhazen_chatbot()

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --f


AttributeError: 'tuple' object has no attribute 'tb_frame'